In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup_backend import setup_backend
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Backend の設定

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend()

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['x', 'measure', 'cz', 'id', 'reset', 'sx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

In [7]:
n_qubits = 4

# Load parameters
with open(f"results/get_param/{n_qubits}Q_param.json", "r") as f:
    param = json.load(f)
C = param["C"]
K = param["K"]
n_features = (
    K + 1
)  # 実数部分の特徴量の数。1 は t=0, つまり定数項。虚数も含んだ特徴量の数は 2K+1

print(f"C: {C}")
print(f"K: {K}")

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q_optimal_n_step.json", orient="records"
)
times = df_n_step["t"].values
print(f"times: {times}")
display(df_n_step)

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q_dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

C: 9
K: 8
times: [0.         0.34906585 0.6981317  1.04719755 1.3962634  1.74532925
 2.0943951  2.44346095 2.7925268 ]


,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.349066,1,0.974201
2,0.698132,2,0.950947
3,1.047198,2,0.946502
4,1.396263,3,0.923480
5,1.745329,3,0.911794
6,2.094395,4,0.892342
7,2.443461,5,0.876343
8,2.792527,5,0.860245


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.5479120971, -0.12224312050000001, 0.7171958...",13.899247,13.899220,2.726160e-05
1,1,"[0.3947360581, -0.8116453042, 0.9512447033]",28.586491,28.585213,1.278298e-03
2,2,"[0.5222794040000001, 0.5721286106, -0.7437727346]",2.653078,2.653082,3.712600e-06
3,3,"[-0.0992281242, -0.2584039515, 0.8535299777]",7.222994,7.222891,1.023508e-04
4,4,"[0.2877302402, 0.6455232265, -0.11317160230000...",0.865096,0.865078,1.800550e-05
5,5,"[-0.5455225564, 0.109169574, -0.8723654878]",1.138947,1.138950,2.786300e-06
6,6,"[0.655262344, 0.2633287982, 0.5161754802]",9.060702,9.060659,4.298700e-05
7,7,"[-0.29094806370000004, 0.9413960488, 0.7862422...",4.366950,4.367027,7.663200e-05
8,8,"[0.5567669941, -0.6107225843, -0.0665579925]",4.566096,4.566077,1.851980e-05
9,9,"[-0.9123924684, -0.6914210159, 0.3660979065]",3.720388,3.720380,8.204800e-06


# Calculate Fourier feature

## Exact (No Trotterization)
Exact means that we compute the matrix exponential of the Hamiltonian, not Trotterized.

In [8]:
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}


for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModel(n_qubits, G)

    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        final_state_phase0, _ = heisenberg.exact_simulation(times[k], phase=0)
        final_state_phase1, _ = heisenberg.exact_simulation(times[k], phase=1)
        final_state_phase2, _ = heisenberg.exact_simulation(times[k], phase=2)
        final_state_phase3, _ = heisenberg.exact_simulation(times[k], phase=3)

        # "00...00" がなければ確率 0 を取得
        prob_phase0 = final_state_phase0.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase1 = final_state_phase1.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase2 = final_state_phase2.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase3 = final_state_phase3.probabilities_dict().get("0" * n_qubits, 0)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_exact = pd.DataFrame(data, columns=columns)
display(df_exact)

# Save the features
path = f"results/fourier_feature_exact/{n_qubits}Q"
df_exact.to_json(f"{path}/features.json", orient="records", indent=4)

# Save the probabilities, for reference
with open(f"{path}/probs_phase0.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/55
Calculating features for sample 1/55
Calculating features for sample 2/55
Calculating features for sample 3/55
Calculating features for sample 4/55
Calculating features for sample 5/55
Calculating features for sample 6/55
Calculating features for sample 7/55
Calculating features for sample 8/55
Calculating features for sample 9/55
Calculating features for sample 10/55
Calculating features for sample 11/55
Calculating features for sample 12/55
Calculating features for sample 13/55
Calculating features for sample 14/55
Calculating features for sample 15/55
Calculating features for sample 16/55
Calculating features for sample 17/55
Calculating features for sample 18/55
Calculating features for sample 19/55
Calculating features for sample 20/55
Calculating features for sample 21/55
Calculating features for sample 22/55
Calculating features for sample 23/55
Calculating features for sample 24/55
Calculating features for sample 25/55
Calculating features f

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,1.0,0.718041,0.383606,0.195441,0.340886,-0.061805,0.044172,-0.059755,-0.029632,-0.207918,0.171675,-0.599948,0.220433,-0.802551,-0.096272,-0.490669,-0.402475,13.899247
1,1,1.0,0.533219,0.556411,-0.176896,0.330914,-0.190092,-0.225399,0.255385,-0.298390,0.379184,-0.034141,0.197245,-0.017423,0.179823,-0.100772,0.162744,0.154607,28.586491
2,2,1.0,0.778346,-0.246211,0.301691,-0.343953,-0.040665,-0.286528,-0.018982,-0.205568,0.232292,-0.226135,0.357286,-0.330143,0.130089,-0.371267,-0.320885,-0.229954,2.653078
3,3,1.0,0.771926,0.295819,0.242453,0.300258,-0.238066,-0.049999,-0.378689,-0.537668,-0.151334,-0.827320,0.207169,-0.716856,0.387829,-0.285992,0.243577,0.167489,7.222994
4,4,1.0,0.963873,-0.163293,0.863299,-0.319218,0.719016,-0.457295,0.557826,-0.563322,0.404492,-0.622343,0.275721,-0.624241,0.178362,-0.569336,0.111796,-0.471004,0.865096
5,5,1.0,0.652979,-0.392155,0.091830,-0.237883,-0.061567,0.099905,-0.026271,0.051280,-0.279651,-0.158154,-0.556160,-0.021664,-0.312819,0.214764,0.192527,-0.045099,1.138947
6,6,1.0,0.801611,0.252730,0.416358,0.227315,0.199764,0.024042,0.200110,-0.014849,0.134899,0.227875,-0.180995,0.480511,-0.546562,0.444028,-0.624683,0.188821,9.060702
7,7,1.0,0.831500,-0.164043,0.476650,-0.340292,0.220555,-0.422985,0.190423,-0.288624,0.285566,0.003609,0.367036,0.211216,0.419182,0.156153,0.486711,-0.076606,4.366950
8,8,1.0,0.855184,0.356904,0.484418,0.563724,0.050315,0.535060,-0.258042,0.286645,-0.308841,-0.072210,-0.084979,-0.386555,0.310219,-0.522861,0.700787,-0.425811,4.566096
9,9,1.0,0.783503,0.073778,0.339573,0.228951,0.056574,0.356156,0.097964,0.268420,0.292518,-0.018056,0.402635,-0.237244,0.390137,-0.168979,0.364223,0.084275,3.720388
